In [1]:
# The goal is to demonstrate how services.xml variants work with single-node deployments with variations on which schemas are deployed.
# Start 3 Vespa dockers on separate ports
# Deploy to (1) with 'deploy:instance=local1', to another with 'deploy:instance=local2',
# List which indices were created by executing ls commands inside the docker containers
# Mention what happens if there is no `deploy:instance="default"` provided (the 3rd container)
# Shutdown all 3 docker images

In [2]:
%%bash
docker run \
  --detach \
  --rm \
  --name vespa-demo-1 \
  --publish 0.0.0.0:8080:8080 \
  --publish 0.0.0.0:19050:19050 \
  --publish 0.0.0.0:19071:19071 \
  vespaengine/vespa:8.625.17

a97e76343bc5cdbfc6a0e7b2cb37907f6d5acf7619fd40b6cfa13fda7a448bbb


In [3]:
%%bash
docker run \
  --detach \
  --rm \
  --name vespa-demo-2 \
  --publish 0.0.0.0:8081:8080 \
  --publish 0.0.0.0:19051:19050 \
  --publish 0.0.0.0:19072:19071 \
  vespaengine/vespa:8.625.17

c44417ff262eb091cf3b88ebcfe84cdb569ea18502043791cabda2af3e4a7404


In [4]:
%%bash
docker run \
  --detach \
  --rm \
  --name vespa-demo-3 \
  --publish 0.0.0.0:8082:8080 \
  --publish 0.0.0.0:19052:19050 \
  --publish 0.0.0.0:19073:19071 \
  vespaengine/vespa:8.625.17

2849ea9ceb0c8e8521cf66ed2f1ac760730a4084d453d1cdec5d0eb284e5bcd3


In [5]:
# Manually create an application package by zipping the files
!zip -r application.zip .

  adding: multi-host/ (stored 0%)
  adding: multi-host/docker-compose.yml (deflated 14%)
  adding: validation-overrides.xml (deflated 22%)
  adding: services.xml (deflated 61%)
  adding: schemas/ (stored 0%)
  adding: schemas/doc2.sd (deflated 34%)
  adding: schemas/doc1.sd (deflated 33%)
  adding: README.md (deflated 78%)
  adding: .vespaignore (deflated 23%)
  adding: session.ipynb (deflated 69%)


In [6]:
%%bash
# Deploy to the vespa-demo-1 as `instance=local-1`
curl -X POST \
        --header "Content-Type:application/zip" \
                 --data-binary @application.zip \
    "http://localhost:19071/application/v2/tenant/default/prepareandactivate?instance=local-1"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5426    0   322  100  5104    286   4536  0:00:01  0:00:01 --:--:--  4823897


{"log":[],"message":"Session 2 for tenant 'default' prepared and activated.","session-id":"2","activated":true,"tenant":"default","url":"http://localhost:19071/application/v2/tenant/default/application/default/environment/prod/region/default/instance/local-1","configChangeActions":{"restart":[],"refeed":[],"reindex":[]}}

In [13]:
%%bash
# Deploy to the vespa-demo-2 as `instance=local-2`
curl -X POST \
        --header "Content-Type:application/zip" \
                 --data-binary @application.zip \
    "http://localhost:19072/application/v2/tenant/default/prepareandactivate?instance=local-2"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5426    0   322  100  5104    344   5468 --:--:-- --:--:-- --:--:--  5809  100  5104    344   5468 --:--:-- --:--:-- --:--:--  5809


{"log":[],"message":"Session 2 for tenant 'default' prepared and activated.","session-id":"2","activated":true,"tenant":"default","url":"http://localhost:19071/application/v2/tenant/default/application/default/environment/prod/region/default/instance/local-2","configChangeActions":{"restart":[],"refeed":[],"reindex":[]}}

In [8]:
%%bash
# Deploy to the vespa-demo-3 without specifying instance
# Also, note that vespa-cli could be used, but let's use curl for consistency
curl -X POST \
        --header "Content-Type:application/zip" \
                 --data-binary @application.zip \
    "http://localhost:19071/application/v2/tenant/default/prepareandactivate"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5247    0   143  100  5104    817  29177 --:--:-- --:--:-- --:--:-- 30155


{"error-code":"INVALID_APPLICATION_PACKAGE","message":"Invalid application: The <documents> element is mandatory in content cluster 'content'"}

In [17]:
# ^ see that the application fails to deploy because no `<documents>` tag is provided.
# This is because an implicit `instance=default` is used by the configserver if there a siblings that has more specific variants.

In [10]:
# Check which schemas are deployed to the instance=local-1
!docker exec vespa-demo-1 sh -c "ls /opt/vespa/var/db/vespa/search/cluster.content/n0/documents"

doc1


In [11]:
# As expected, only the `doc1`

In [14]:
# Check which schemas are deployed to the instance=local-2
!docker exec vespa-demo-2 sh -c "ls /opt/vespa/var/db/vespa/search/cluster.content/n0/documents"

doc1
doc2


In [15]:
# We have `doc1` and the `doc2` schemas deployed. Nice.

In [16]:
!docker kill vespa-demo-3
!docker kill vespa-demo-2
!docker kill vespa-demo-1

vespa-demo-3
vespa-demo-2
vespa-demo-1
